# Crypto Price Shear Mean Reversion

Learn how to backtest a high-frequency price shear mean reversion algorithmic trading system on multiple assets with multiple timeframes using Python and Pandas.

---

In this post, we'll download minutely data from Kaggle, resample the data into 5 minute and daily bars, and analyze the data for a price shear.

A price shear is simply an excessive move to the upside or downside relative to historical bars. We expect algos and traders to capture a profit from this move causing the price to mean revert.


#### Download Links:
- [Python Crypto Algo Trading Strategy YouTube Video](https://youtu.be/Z1UnatwCwRU)
- [Kaggle Minutely Cryptocurrency Price History](https://www.kaggle.com/sudalairajkumar/cryptocurrencypricehistory)

## 1. Get Imports

In [1]:
import datetime as dt
import numpy as np
import pandas as pd

## 2. Import Universe

In [2]:
from zipfile import ZipFile
zf = ZipFile('/home/leosmigel/Downloads/archive.zip')
cols = ['time', 'open', 'high', 'low', 'close', 'volume']

In [3]:
dfs = pd.concat({text_file.filename.split('.')[0]: pd.read_csv(zf.open(text_file.filename),
                                                              usecols=cols)
                
                for text_file in zf.infolist()
                if text_file.filename.endswith('.csv')
                })
dfs

time     open    close     high      low  \
1inchusd 0       1627916520000  2.38090  2.38240  2.38240  2.38090   
         1       1627916580000  2.37750  2.38730  2.38730  2.37740   
         2       1627916700000  2.37980  2.37980  2.37980  2.37980   
         3       1627921080000  2.37940  2.37930  2.37940  2.37930   
         4       1627921140000  2.37930  2.37990  2.37990  2.37930   
...                        ...      ...      ...      ...      ...   
zrxusd   407696  1628630940000  0.97622  0.97622  0.97622  0.97622   
         407697  1628631420000  0.97525  0.97377  0.97525  0.97377   
         407698  1628631900000  0.97182  0.97182  0.97182  0.97182   
         407699  1628632080000  0.96957  0.97008  0.97008  0.96957   
         407700  1628632680000  0.97205  0.97205  0.97205  0.97205   

                      volume  
1inchusd 0         29.697338  
         1        115.688719  
         2          0.041380  
         3       1296.114682  
         4          0.026510  
...                      ...  
zrxusd   407696    59.570146  
         407697   228.824921  
         407698    60.145555  
         407699  4231.639947  
         407700     7.550140  

[61784313 rows x 6 columns]

In [4]:
df = dfs.droplevel(1).reset_index().rename(columns={'index':'ticker'})
df = df[df['ticker'].str.contains('usd')]
df['date'] = pd.to_datetime(df['time'], unit='ms')
df = df.sort_values(by=['date','ticker'])
df = df.drop(columns='time')
df = df.set_index(['date','ticker'])
df = df['2020-07-01':'2020-12-31']
df

open        close          high           low  \
date       ticker                                                          
2020-07-01 btcusd   9150.646722   9147.30000   9150.646722   9147.300000   
           btgusd     10.403000     10.40300     10.403000     10.403000   
           eosusd      2.370600      2.37060      2.370600      2.370600   
           ethusd    225.880000    225.69000    225.880000    225.671073   
           gotusd      0.042020      0.04380      0.043800      0.042020   
...                         ...          ...           ...           ...   
2020-12-31 xlmusd      0.131800      0.13180      0.131800      0.131800   
           xrpusd      0.211350      0.21094      0.211350      0.209870   
           xtzusd      1.988200      1.98820      1.988200      1.988200   
           yfiusd  21792.000000  21792.00000  21792.000000  21792.000000   
           zecusd     64.057000     64.09400     64.094000     64.057000   

                         volume  
date       ticker                
2020-07-01 btcusd      1.452704  
           btgusd    141.000000  
           eosusd    136.577291  
           ethusd     12.266386  
           gotusd    160.000000  
...                         ...  
2020-12-31 xlmusd     91.628590  
           xrpusd  50985.665741  
           xtzusd    125.735553  
           yfiusd      0.148400  
           zecusd      1.263063  

[3842732 rows x 5 columns]

# 3. Create Indicators

In [5]:
def wwma(values, n):
    """
     J. Welles Wilder's EMA 
    """
    x = values.groupby('ticker').ewm(alpha=1/n, adjust=False, min_periods=n).mean().droplevel(0)
    return x
    

def atr(df, n=12):
    data = df.copy()
    high = data['high']
    low = data['low']
    close = data['close']
    data['tr0'] = abs(high - low)
    data['tr1'] = abs(high - close.groupby('ticker').shift())
    data['tr2'] = abs(low - close.groupby('ticker').shift())
    tr = data[['tr0', 'tr1', 'tr2']].max(axis=1)
    #return tr
    atr = wwma(tr, n)
    atr.name = 'atr'
    return atr

In [6]:
ema12 = lambda x: x.ewm(span=12, min_periods=12, adjust=False, ignore_na=False).mean()
ema26 = lambda x: x.ewm(span=26, min_periods=26, adjust=False, ignore_na=False).mean()

## 4. Resample Timeframes

In [7]:
bars1m = df
bars1m = bars1m.reset_index().set_index('date').groupby('ticker').resample('1min').last().droplevel(0)
bars1m.loc[:, bars1m.columns[:-1]] = bars1m[bars1m.columns[:-1]].ffill()
bars1m.loc[:, 'volume'] = bars1m['volume'].fillna(value=0.0)
bars1m = bars1m.reset_index().set_index(['date','ticker'])
bars1m

,,open,close,high,low,volume
date,ticker,,,,,
2020-08-06 10:11:00,adausd,0.14270,0.14270,0.14270,0.14270,10.0000
2020-08-06 10:12:00,adausd,0.14270,0.14270,0.14270,0.14270,0.0000
2020-08-06 10:13:00,adausd,0.14270,0.14270,0.14270,0.14270,0.0000
2020-08-06 10:14:00,adausd,0.14270,0.14270,0.14270,0.14270,0.0000
2020-08-06 10:15:00,adausd,0.14251,0.14251,0.14251,0.14251,7557.1124
...,...,...,...,...,...,...
2020-12-30 23:51:00,zrxusd,0.36008,0.36008,0.36008,0.36008,0.0000
2020-12-30 23:52:00,zrxusd,0.36008,0.36008,0.36008,0.36008,0.0000
2020-12-30 23:53:00,zrxusd,0.36008,0.36008,0.36008,0.36008,0.0000


In [8]:
aggdict = {
    'open':'first',
    'high':'max',
    'low':'min',
    'close':'last',
    'volume':'sum'
}

bars5m = bars1m.groupby([pd.Grouper(level=0, freq='5min', label='right'),
                         pd.Grouper(level=1)]).agg(aggdict)
bars5m['ema12'] = bars5m.groupby('ticker')['close'].apply(ema12)
bars5m['ema26'] = bars5m.groupby('ticker')['close'].apply(ema26)
bars5m = bars5m.merge(atr(bars5m), on=['date','ticker'])
bars5m = bars5m.dropna(subset=['ema26'])
bars5m.columns = [c + '_5m' for c in bars5m.columns]
bars5m

open_5m       high_5m       low_5m  \
date                ticker                                            
2020-07-01 02:10:00 btcusd   9134.250244   9134.250244   9133.80000   
                    btgusd     10.202000     10.202000     10.18200   
                    daiusd      1.008900      1.008900      1.00890   
                    dshusd     67.981000     67.981000     67.98100   
                    eosusd      2.364500      2.364500      2.36450   
...                                  ...           ...          ...   
2020-12-31 00:05:00 xlmusd      0.131800      0.131800      0.13180   
                    xrpusd      0.211350      0.211350      0.20987   
                    xtzusd      1.988200      1.988200      1.98820   
                    yfiusd  21792.000000  21792.000000  21792.00000   
                    zecusd     64.057000     64.094000     64.05700   

                               close_5m     volume_5m      ema12_5m  \
date                ticker                                            
2020-07-01 02:10:00 btcusd   9133.90000      2.216256   9130.959311   
                    btgusd     10.18200      0.000000     10.233967   
                    daiusd      1.00890      0.000000      1.008261   
                    dshusd     67.98100      0.000000     67.928498   
                    eosusd      2.36450      0.000000      2.363894   
...                                 ...           ...           ...   
2020-12-31 00:05:00 xlmusd      0.13180     91.628590      0.131154   
                    xrpusd      0.21094  50985.665741      0.211566   
                    xtzusd      1.98820    125.735553      1.986940   
                    yfiusd  21792.00000      0.148400  21850.168026   
                    zecusd     64.09400      1.263063     63.942956   

                                ema26_5m     atr_5m  
date                ticker                           
2020-07-01 02:10:00 btcusd   9128.600825   6.836394  
                    btgusd     10.274364   0.026298  
                    daiusd      1.007497   0.000766  
                    dshusd     67.947234   0.110720  
                    eosusd      2.363118   0.002132  
...                                  ...        ...  
2020-12-31 00:05:00 xlmusd      0.131139   0.000359  
                    xrpusd      0.211520   0.002468  
                    xtzusd      1.987533   0.003237  
                    yfiusd  21855.786949  61.664108  
                    zecusd     63.951183   0.162526  

[6399195 rows x 8 columns]

In [9]:
bars1d = bars1m.groupby([pd.Grouper(level=0, freq='1d', label='right'),
                         pd.Grouper(level=1)]).agg(aggdict)
bars1d['ema12'] = bars1d.groupby('ticker')['close'].apply(ema12)
bars1d['ema26'] = bars1d.groupby('ticker')['close'].apply(ema26)
bars1d = bars1d.dropna(subset=['ema26'])
bars1d.columns = [c + '_1d' for c in bars1d.columns]
bars1d

open_1d      high_1d       low_1d     close_1d  \
date       ticker                                                       
2020-07-27 algusd      0.32014      0.33017      0.30657      0.32177   
           ampusd      2.68570      2.89900      2.28030      2.84500   
           antusd      2.21700      2.33030      2.04150      2.13510   
           atousd      3.91550      3.91550      3.72430      3.83000   
           avtusd      0.13623      0.13623      0.13623      0.13623   
...                        ...          ...          ...          ...   
2021-01-01 xlmusd      0.13180      0.13180      0.13180      0.13180   
           xrpusd      0.21135      0.21135      0.20987      0.21094   
           xtzusd      1.98820      1.98820      1.98820      1.98820   
           yfiusd  21792.00000  21792.00000  21792.00000  21792.00000   
           zecusd     64.05700     64.09400     64.05700     64.09400   

                       volume_1d      ema12_1d      ema26_1d  
date       ticker                                             
2020-07-27 algusd  190186.672185      0.323712      0.298166  
           ampusd  488589.065761      2.531786      2.536300  
           antusd   73597.903674      1.735006      1.575232  
           atousd    4656.457442      3.909348      3.690261  
           avtusd       0.000000      0.145681      0.147010  
...                          ...           ...           ...  
2021-01-01 xlmusd      91.628590      0.146912      0.152961  
           xrpusd   50985.665741      0.312610      0.392278  
           xtzusd     125.735553      2.061433      2.127737  
           yfiusd       0.148400  23117.390976  23849.996498  
           zecusd       1.263063     66.064338     67.434965  

[19034 rows x 7 columns]

## 5. Merge Timeframes

In [10]:
print(bars1d.index[0][0])
print(bars5m.index[0][0])
print(bars1d.index[-1][0])
print(bars5m.index[-1][0])

2020-07-27 00:00:00
2020-07-01 02:10:00
2021-01-01 00:00:00
2020-12-31 00:05:00


In [11]:
bars5m = bars5m.reset_index()
bars1d = bars1d.reset_index()

In [12]:
bars5m = bars5m.loc[bars5m['date'] >= bars1d['date'].min(), :]
bars1d = bars1d.loc[bars1d['date'] <= bars5m['date'].max(), :]
bars5m = bars5m.loc[bars5m['date'] <= bars1d['date'].max(), :]

In [13]:
print(bars1d.date.iloc[0])
print(bars5m.date.iloc[0])
print(bars1d.date.iloc[-1])
print(bars5m.date.iloc[-1])

2020-07-27 00:00:00
2020-07-27 00:00:00
2020-12-31 00:00:00
2020-12-31 00:00:00


In [14]:
bars = bars5m.merge(bars1d, on=['date','ticker'], how='left')
bars[bars1d.columns] = bars.groupby('ticker')[bars1d.columns].transform(lambda x: x.ffill())
bars = bars.set_index(['date','ticker'])
bars

open_5m       high_5m        low_5m      close_5m  \
date       ticker                                                           
2020-07-27 algusd      0.321770      0.321770      0.321770      0.321770   
           ampusd      2.849900      2.849900      2.845000      2.845000   
           antusd      2.230000      2.230000      2.135100      2.135100   
           astusd      0.067292      0.067292      0.067292      0.067292   
           atousd      3.830000      3.830000      3.830000      3.830000   
...                         ...           ...           ...           ...   
2020-12-31 yfiusd  21902.000000  21902.000000  21770.000000  21770.000000   
           zbtusd      0.254210      0.254240      0.254210      0.254240   
           zecusd     64.063000     64.172000     63.991000     64.172000   
           zilusd      0.078197      0.078197      0.077227      0.077227   
           zrxusd      0.360390      0.360390      0.360390      0.360390   

                      volume_5m      ema12_5m      ema26_5m        atr_5m  \
date       ticker                                                           
2020-07-27 algusd      0.000000      0.319050      0.316583  7.210629e-04   
           ampusd   2376.327500      2.841257      2.839157  2.049029e-02   
           antusd      0.000000      2.176580      2.208885  4.849922e-02   
           astusd      0.000000      0.067292      0.067292  8.778113e-15   
           atousd      0.000000      3.833479      3.837965  7.000615e-03   
...                         ...           ...           ...           ...   
2020-12-31 yfiusd      0.004800  21860.744030  21860.889905  6.526994e+01   
           zbtusd     14.693540      0.254141      0.254123  4.226916e-05   
           zecusd     62.826867     63.915493     63.939758  1.668470e-01   
           zilusd  13447.600000      0.078438      0.078661  2.447524e-04   
           zrxusd     33.142800      0.357946      0.356835  6.577515e-04   

                        open_1d      high_1d        low_1d      close_1d  \
date       ticker                                                          
2020-07-27 algusd      0.320140      0.33017      0.306570      0.321770   
           ampusd      2.685700      2.89900      2.280300      2.845000   
           antusd      2.217000      2.33030      2.041500      2.135100   
           astusd           NaN          NaN           NaN           NaN   
           atousd      3.915500      3.91550      3.724300      3.830000   
...                         ...          ...           ...           ...   
2020-12-31 yfiusd  22199.000000  22727.00000  21432.000000  21770.000000   
           zbtusd      0.254490      0.25751      0.251260      0.254240   
           zecusd     65.453000     66.13300     62.745000     64.172000   
           zilusd      0.082297      0.08410      0.070012      0.077227   
           zrxusd      0.365120      0.37063      0.348830      0.360390   

                      volume_1d      ema12_1d      ema26_1d  
date       ticker                                            
2020-07-27 algusd  1.901867e+05      0.323712      0.298166  
           ampusd  4.885891e+05      2.531786      2.536300  
           antusd  7.359790e+04      1.735006      1.575232  
           astusd           NaN           NaN           NaN  
           atousd  4.656457e+03      3.909348      3.690261  
...                         ...           ...           ...  
2020-12-31 yfiusd  7.572732e+01  23358.371154  24014.636218  
           zbtusd  5.336957e+03      0.259251      0.263041  
           zecusd  1.432417e+04     66.422581     67.702242  
           zilusd  1.989765e+06      0.068241      0.054749  
           zrxusd  2.623325e+05      0.371207      0.379854  

[5574959 rows x 15 columns]

## 6. Create Signal

In [15]:
bars['shear_bear']  = (np.abs(bars['close_5m'] - bars['ema12_5m']) > (bars['atr_5m'] * 2.5)) & \
(bars['close_5m'] < bars['ema12_5m'])

In [16]:
bars['position'] = np.nan
bars['position'] = np.where(((bars['shear_bear'] == True) &
                            (bars.groupby('ticker')['shear_bear'].shift() == False)), 1, 0)
bars['position'] = bars.groupby('ticker')['position'].shift()
bars

open_5m       high_5m        low_5m      close_5m  \
date       ticker                                                           
2020-07-27 algusd      0.321770      0.321770      0.321770      0.321770   
           ampusd      2.849900      2.849900      2.845000      2.845000   
           antusd      2.230000      2.230000      2.135100      2.135100   
           astusd      0.067292      0.067292      0.067292      0.067292   
           atousd      3.830000      3.830000      3.830000      3.830000   
...                         ...           ...           ...           ...   
2020-12-31 yfiusd  21902.000000  21902.000000  21770.000000  21770.000000   
           zbtusd      0.254210      0.254240      0.254210      0.254240   
           zecusd     64.063000     64.172000     63.991000     64.172000   
           zilusd      0.078197      0.078197      0.077227      0.077227   
           zrxusd      0.360390      0.360390      0.360390      0.360390   

                      volume_5m      ema12_5m      ema26_5m        atr_5m  \
date       ticker                                                           
2020-07-27 algusd      0.000000      0.319050      0.316583  7.210629e-04   
           ampusd   2376.327500      2.841257      2.839157  2.049029e-02   
           antusd      0.000000      2.176580      2.208885  4.849922e-02   
           astusd      0.000000      0.067292      0.067292  8.778113e-15   
           atousd      0.000000      3.833479      3.837965  7.000615e-03   
...                         ...           ...           ...           ...   
2020-12-31 yfiusd      0.004800  21860.744030  21860.889905  6.526994e+01   
           zbtusd     14.693540      0.254141      0.254123  4.226916e-05   
           zecusd     62.826867     63.915493     63.939758  1.668470e-01   
           zilusd  13447.600000      0.078438      0.078661  2.447524e-04   
           zrxusd     33.142800      0.357946      0.356835  6.577515e-04   

                        open_1d      high_1d        low_1d      close_1d  \
date       ticker                                                          
2020-07-27 algusd      0.320140      0.33017      0.306570      0.321770   
           ampusd      2.685700      2.89900      2.280300      2.845000   
           antusd      2.217000      2.33030      2.041500      2.135100   
           astusd           NaN          NaN           NaN           NaN   
           atousd      3.915500      3.91550      3.724300      3.830000   
...                         ...          ...           ...           ...   
2020-12-31 yfiusd  22199.000000  22727.00000  21432.000000  21770.000000   
           zbtusd      0.254490      0.25751      0.251260      0.254240   
           zecusd     65.453000     66.13300     62.745000     64.172000   
           zilusd      0.082297      0.08410      0.070012      0.077227   
           zrxusd      0.365120      0.37063      0.348830      0.360390   

                      volume_1d      ema12_1d      ema26_1d  shear_bear  \
date       ticker                                                         
2020-07-27 algusd  1.901867e+05      0.323712      0.298166       False   
           ampusd  4.885891e+05      2.531786      2.536300       False   
           antusd  7.359790e+04      1.735006      1.575232       False   
           astusd           NaN           NaN           NaN       False   
           atousd  4.656457e+03      3.909348      3.690261       False   
...                         ...           ...           ...         ...   
2020-12-31 yfiusd  7.572732e+01  23358.371154  24014.636218       False   
           zbtusd  5.336957e+03      0.259251      0.263041       False   
           zecusd  1.432417e+04     66.422581     67.702242       False   
           zilusd  1.989765e+06      0.068241      0.054749        True   
           zrxusd  2.623325e+05      0.371207      0.379854       False   

                   position  
date       ticker        

In [17]:
btc = bars.swaplevel(1,0).xs('btcusd')
btc['2020-12-27 20:45:00':].head(20)

,open_5m,high_5m,low_5m,close_5m,volume_5m,ema12_5m,ema26_5m,atr_5m,open_1d,high_1d,low_1d,close_1d,volume_1d,ema12_1d,ema26_1d,shear_bear,position
date,,,,,,,,,,,,,,,,,
2020-12-27 20:45:00,26602.000000,26690.000000,26595.371600,26636.000000,16.275885,26812.093307,26879.028867,105.319060,24720.0882,26723.0,24489.0,26413.0,13149.603522,23250.671744,21450.679928,False,0.0
2020-12-27 20:50:00,26636.000000,26636.000000,26220.000000,26254.000000,444.598148,26726.232798,26832.730433,131.209138,24720.0882,26723.0,24489.0,26413.0,13149.603522,23250.671744,21450.679928,True,0.0
2020-12-27 20:55:00,26254.000000,26401.000000,26241.774600,26242.000000,64.883661,26651.735445,26788.972623,133.543826,24720.0882,26723.0,24489.0,26413.0,13149.603522,23250.671744,21450.679928,True,1.0
2020-12-27 21:00:00,26242.000000,26349.000000,26150.000000,26182.304127,165.164127,26579.515242,26744.034216,138.998508,24720.0882,26723.0,24489.0,26413.0,13149.603522,23250.671744,21450.679928,True,0.0
2020-12-27 21:05:00,26174.000000,26314.000000,26110.000000,26151.363700,160.313818,26513.645774,26700.132696,144.415299,24720.0882,26723.0,24489.0,26413.0,13149.603522,23250.671744,21450.679928,True,0.0
2020-12-27 21:10:00,26137.000000,26282.000000,26110.000000,26222.000000,45.850048,26468.777193,26664.715459,146.714024,24720.0882,26723.0,24489.0,26413.0,13149.603522,23250.671744,21450.679928,False,0.0
2020-12-27 21:15:00,26230.000000,26254.000000,26121.000000,26162.000000,22.210182,26421.580702,26627.477277,145.571188,24720.0882,26723.0,24489.0,26413.0,13149.603522,23250.671744,21450.679928,False,0.0
2020-12-27 21:20:00,26161.000000,26190.000000,25827.212600,25880.012512,182.701262,26338.262519,26572.109517,163.672539,24720.0882,26723.0,24489.0,26413.0,13149.603522,23250.671744,21450.679928,True,0.0
2020-12-27 21:25:00,25870.000000,26147.000000,25813.000000,26101.000000,149.432722,26301.760593,26537.212515,177.866494,24720.0882,26723.0,24489.0,26413.0,13149.603522,23250.671744,21450.679928,False,1.0


## 7. Examine Results

In [18]:
fee = 0.0
fee = np.log(0.0018)
bars['close_5m_log'] = bars['close_5m'].apply(np.log)
bars['bar_return_log'] = bars.groupby('ticker')['close_5m_log'].diff()
bars = bars.dropna(subset=['bar_return_log'])
bars = bars.join(bars.groupby('date')['position'].sum(), on='date', rsuffix='_count')

bars['r'] = bars['bar_return_log'] * bars['position'] / bars['position_count']
bars['fees'] = np.where(bars['position'] != 0, fee, 0)
bars


open_5m       high_5m        low_5m  \
date                ticker                                             
2020-07-27 00:05:00 algusd      0.321770      0.321770      0.321770   
                    ampusd      2.845000      2.845000      2.845000   
                    antusd      2.230000      2.230000      2.135100   
                    astusd      0.067292      0.067292      0.067292   
                    atousd      3.830000      3.830000      3.830000   
...                                  ...           ...           ...   
2020-12-31 00:00:00 yfiusd  21902.000000  21902.000000  21770.000000   
                    zbtusd      0.254210      0.254240      0.254210   
                    zecusd     64.063000     64.172000     63.991000   
                    zilusd      0.078197      0.078197      0.077227   
                    zrxusd      0.360390      0.360390      0.360390   

                                close_5m     volume_5m      ema12_5m  \
date                ticker                                             
2020-07-27 00:05:00 algusd      0.321770      0.000000      0.319469   
                    ampusd      2.845000      0.000000      2.841833   
                    antusd      2.135100      0.000000      2.170199   
                    astusd      0.067292      0.000000      0.067292   
                    atousd      3.830000      0.000000      3.832944   
...                                  ...           ...           ...   
2020-12-31 00:00:00 yfiusd  21770.000000      0.004800  21860.744030   
                    zbtusd      0.254240     14.693540      0.254141   
                    zecusd     64.172000     62.826867     63.915493   
                    zilusd      0.077227  13447.600000      0.078438   
                    zrxusd      0.360390     33.142800      0.357946   

                                ema26_5m        atr_5m       open_1d  \
date                ticker                                             
2020-07-27 00:05:00 algusd      0.316968  6.609743e-04      0.320140   
                    ampusd      2.839590  1.878277e-02      2.685700   
                    antusd      2.203420  5.236595e-02      2.217000   
                    astusd      0.067292  8.046604e-15           NaN   
                    atousd      3.837375  6.417231e-03      3.915500   
...                                  ...           ...           ...   
2020-12-31 00:00:00 yfiusd  21860.889905  6.526994e+01  22199.000000   
                    zbtusd      0.254123  4.226916e-05      0.254490   
                    zecusd     63.939758  1.668470e-01     65.453000   
                    zilusd      0.078661  2.447524e-04      0.082297   
                    zrxusd      0.356835  6.577515e-04      0.365120   

                                high_1d  ...     volume_1d      ema12_1d  \
date                ticker               ...                               
2020-07-27 00:05:00 algusd      0.33017  ...  1.901867e+05      0.323712   
                    ampusd      2.89900  ...  4.885891e+05      2.531786   
                    antusd      2.33030  ...  7.359790e+04      1.735006   
                    astusd          NaN  ...           NaN           NaN   
                    atousd      3.91550  ...  4.656457e+03      3.909348   
...                                 ...  ...           ...           ...   
2020-12-31 00:00:00 yfiusd  22727.00000  ...  7.572732e+01  23358.371154   
                    zbtusd      0.25751  ...  5.336957e+03      0.259251   
                    zecusd     66.13300  ...  1.432417e+04     66.422581   
                    zilusd      0.08410  ...  1.989765e+06      0.068241   
                    zrxusd      0.37063  ...  2.623325e+05      0.371207   

                                ema26_1d  shear_bear  position  close_5m_log  \
date                ticker                                                     
2020-07-27 00:05:00 algusd      0.298166       False       0.0     -1.133

In [19]:
bars[bars['position'] == 1]

open_5m     high_5m      low_5m    close_5m  \
date                ticker                                                   
2020-07-27 00:15:00 pnkusd    0.049473    0.049473    0.049472    0.049472   
                    xtzusd    2.944700    2.950000    2.942900    2.950000   
2020-07-27 00:25:00 gntusd    0.060000    0.060000    0.060000    0.060000   
2020-07-27 00:30:00 gotusd    0.055000    0.055000    0.055000    0.055000   
2020-07-27 00:40:00 manusd    0.075000    0.075000    0.075000    0.075000   
...                                ...         ...         ...         ...   
2020-12-30 23:25:00 bsvusd  166.570000  166.580000  166.380000  166.380000   
                    kncusd    0.801360    0.801360    0.798210    0.798210   
2020-12-30 23:30:00 ksmusd   61.890000   61.954000   61.890000   61.954000   
2020-12-30 23:35:00 btgusd    8.365300    8.365300    8.365300    8.365300   
2020-12-30 23:50:00 sntusd    0.031184    0.031184    0.031184    0.031184   

                              volume_5m    ema12_5m    ema26_5m    atr_5m  \
date                ticker                                                  
2020-07-27 00:15:00 pnkusd     0.000000    0.052690    0.053306  0.000344   
                    xtzusd   510.933712    2.979601    2.986151  0.005766   
2020-07-27 00:25:00 gntusd     0.000000    0.060154    0.060155  0.000050   
2020-07-27 00:30:00 gotusd     0.000000    0.061995    0.063376  0.000746   
2020-07-27 00:40:00 manusd     0.000000    0.081443    0.082715  0.000687   
...                                 ...         ...         ...       ...   
2020-12-30 23:25:00 bsvusd     1.266448  167.073924  167.325812  0.283810   
                    kncusd  1955.000000    0.804318    0.804961  0.000666   
2020-12-30 23:30:00 ksmusd     8.400000   62.267649   62.400507  0.174047   
2020-12-30 23:35:00 btgusd     0.000000    8.402707    8.410843  0.014845   
2020-12-30 23:50:00 sntusd     0.000000    0.031233    0.031246  0.000007   

                               open_1d     high_1d  ...     volume_1d  \
date                ticker                          ...                 
2020-07-27 00:15:00 pnkusd    0.056316    0.056316  ...  8.897491e+03   
                    xtzusd    3.068200    3.118100  ...  1.339461e+06   
2020-07-27 00:25:00 gntusd    0.064700    0.064700  ...  6.136948e+04   
2020-07-27 00:30:00 gotusd    0.050295    0.064770  ...  4.369200e+02   
2020-07-27 00:40:00 manusd    0.075510    0.084000  ...  1.240000e+02   
...                                ...         ...  ...           ...   
2020-12-30 23:25:00 bsvusd  168.010000  168.900000  ...  4.965648e+03   
                    kncusd    0.844960    0.873370  ...  1.118051e+04   
2020-12-30 23:30:00 ksmusd   58.275000   62.467000  ...  1.268222e+03   
2020-12-30 23:35:00 btgusd    8.567800    8.567800  ...  4.001382e+03   
2020-12-30 23:50:00 sntusd    0.031250    0.033278  ...  2.660058e+05   

                              ema12_1d    ema26_1d  shear_bear  position  \
date                ticker                                                 
2020-07-27 00:15:00 pnkusd    0.052999    0.044136        True       1.0   
                    xtzusd    2.995901    2.849993        True       1.0   
2020-07-27 00:25:00 gntusd    0.059241    0.056960        True       1.0   
2020-07-27 00:30:00 gotusd    0.057617    0.055500        True       1.0   
2020-07-27 00:40:00 manusd    0.073102    0.068575        True       1.0   
...                                ...         ...         ...       ...   
2020-12-30 23:25:00 bsvusd  167.446507  169.412062       False       1.0   
                    kncusd    0.845173    0.879494        True       1.0   
2020-12-30 23:30:00 ksmusd   53.659778   51.796203       False       1.0   
2020-12-30 23:35:00 btgusd    8.691705    8.781055        True       1.0   
2020-12-30 23:50:00 sntusd    0.032291    0.033147        True       1.0   

                            close_5m_log  bar_return_log  position_count  \

In [20]:
performance = bars.groupby([pd.Grouper(level=0, freq='1d'),
                            pd.Grouper(level=1)]).agg({
    'close_5m':'last',
    'bar_return_log':'sum',
    'r':'sum',
    'fees':'sum'
})
performance['rf'] = performance['r'] + performance['fees']
performance['tr'] = performance['rf'].cumsum()


In [21]:
performance

close_5m  bar_return_log         r       fees  \
date       ticker                                                      
2020-07-27 algusd      0.287630       -0.112162 -0.014680 -75.839623   
           ampusd      2.274400       -0.223847  0.036910 -44.239780   
           antusd      1.977700       -0.076579 -0.007627 -50.559749   
           astusd      0.048600       -0.325418  0.000000  -6.319969   
           atousd      3.562500       -0.072402  0.017370 -31.599843   
...                         ...             ...       ...        ...   
2020-12-31 yfiusd  21770.000000       -0.005406  0.000000   0.000000   
           zbtusd      0.254240        0.000118  0.000000   0.000000   
           zecusd     64.172000        0.001700  0.000000   0.000000   
           zilusd      0.077227       -0.012482  0.000000   0.000000   
           zrxusd      0.360390        0.000861  0.000000   0.000000   

                          rf             tr  
date       ticker                            
2020-07-27 algusd -75.854304     -75.854304  
           ampusd -44.202870    -120.057174  
           antusd -50.567376    -170.624549  
           astusd  -6.319969    -176.944518  
           atousd -31.582473    -208.526991  
...                      ...            ...  
2020-12-31 yfiusd   0.000000 -376889.948834  
           zbtusd   0.000000 -376889.948834  
           zecusd   0.000000 -376889.948834  
           zilusd   0.000000 -376889.948834  
           zrxusd   0.000000 -376889.948834  

[19431 rows x 6 columns]

In [22]:
if len(performance.index.levels[1].unique()) > 1:
    benchmark = performance.swaplevel(0,1).xs('btcusd')
    performance = performance.groupby('date').last().drop(columns='close_5m')
else:
    performance = performance.droplevel(1)
    benchmark = performance.copy()

benchmark['r'] = benchmark['bar_return_log']
benchmark['tr'] = benchmark['r'].cumsum()
benchmark = benchmark.drop(columns=['fees'])
benchmark
    

,close_5m,bar_return_log,r,rf,tr
date,,,,,
2020-07-27,11018.000000,0.102119,0.102119,-6.318902,0.102119
2020-07-28,10922.000000,-0.008751,-0.008751,-18.958569,0.093367
2020-07-29,11110.000000,0.017066,0.017066,-12.639606,0.110434
2020-07-30,11133.000000,0.002068,0.002068,-44.237569,0.112502
2020-07-31,11345.000000,0.018863,0.018863,-6.320513,0.131365
...,...,...,...,...,...
2020-12-27,26318.000000,-0.003021,-0.003021,-18.951867,0.972841
2020-12-28,27099.154989,0.029249,0.029249,-12.639425,1.002091
2020-12-29,27323.000000,0.008226,0.008226,0.000000,1.010317


In [23]:
import plotly.graph_objects as go
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)
fig = go.Figure()
fig.add_trace(go.Scatter(x=benchmark.index, y=benchmark['tr'], name='Benchmark'))
fig.add_trace(go.Scatter(x=performance.index, y=performance['tr'], name='Strategy'))

fig.update_layout(title=dict(text="Strategy vs. Bitcoin", font=dict(size=24)),
                 legend=dict(font=dict(size=20)),
                 width=800,
                 height=600)

fig.update_xaxes(tickfont=dict(size=20, color="#434"))
fig.update_yaxes(tickfont=dict(size=20, color="#434"))
fig.show()

In [24]:
round(benchmark.iloc[-1]['close_5m'] / benchmark.iloc[0]['close_5m'] -1, 2)

1.62

In [25]:
round(np.exp(benchmark.iloc[-1]['tr'] - benchmark.iloc[0]['tr']) -1, 2)

1.62

In [26]:
round(np.exp(performance.iloc[-1]['tr']) -1, 2)

-1.0